In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json
import gmaps

# Configure gmaps
gmaps.configure(api_key=gkey)

# API Keys
from config import gkey
from config import api_key

NameError: name 'gkey' is not defined

In [ ]:
#Google Places Types: https://developers.google.com/places/web-service/supported_types

#TODO: Can we incorporate this list into the loop so this list can change without changing the code
types = ['bank']#, 'library', 'park', 'liquor_store', 'hospital']

In [ ]:
# Read in the Dallas County Appraisal District (DCAD) property values file

file = "Resources/dcad_combined-Copy1.csv"

#create DataFrame
dcad_df = pd.read_csv(file, usecols=['PROPERTY_ZIPCODE', 'TOT_VAL'])
dcad_df.head(5)

In [ ]:
#add column for 5 digit zipcode
dcad_df['ZIPCODE'] = dcad_df['PROPERTY_ZIPCODE'].astype(str).str[:5]

In [ ]:
#Group Property Values by Zipcode
zip_group = dcad_df.groupby('ZIPCODE')['TOT_VAL']

In [ ]:
zip_group_df = dcad_df.groupby('ZIPCODE')['TOT_VAL'].agg(('count', 'mean'))\
                    .rename(columns={'count':'PropertyValueCount', 'mean':'MeanPropertyValue'})\
                    .sort_values(by='MeanPropertyValue', ascending=False)\
                    .reset_index()

In [ ]:
#Add columns to DataFrame to store business data
zip_group_df["MeanPropertyDollarValue"] = zip_group_df["MeanPropertyValue"].map("${:,.0f}".format)
zip_group_df['Lat'] = "" 
zip_group_df["Lng"] = ""
zip_group_df["City"] = ""
zip_group_df["State"] = ""
zip_group_df = zip_group_df.rename(columns={"ZIPCODE": "Zipcode"})

In [ ]:
#Preview 5 Zipcodes with highest values
zip_group_test_df = zip_group_df.head(5)

In [ ]:
#Get 5 Zipcodes with lowest values
zip_group_df.tail(5)

In [ ]:
# create a params dict that will be updated with new zipcode each iteration
params = {"key": gkey}

# Loop through the zipcode pd's and run a lat/long search for each
for index, row in zip_group_test_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['Zipcode']

    # update address key value to zipcode
    params['address'] = zipcode

    # make request
    zips_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    zips_lat_lng = zips_lat_lng.json()
    #print(json.dumps(zips_lat_lng, indent=4, sort_keys=True))
    
    #loop through address to find city
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'locality':
            zip_group_df.loc[index, "City"] = i['long_name']
    
    #loop though address to find state
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'administrative_area_level_1':
            zip_group_df.loc[index, "State"] = i['short_name']
    zip_group_df.loc[index, "Lat"] = zips_lat_lng["results"][0]["geometry"]["location"]["lat"]
    zip_group_df.loc[index, "Lng"] = zips_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
zip_group_df

In [ ]:
# params dictionary to update each iteration
for each_type in types:
    params = {
        #3 mi radius. A Zipcode is not returned in the results, so we cannot 
        #match against our zipcode without doing a reverse lookup for every result
        "radius": 4828,
        "types": each_type,
        "keyword": "bank",
        "key": gkey
    }
    
    #variables for the specific column names for the business type we are searching
    count_column = f"{each_type}_count"
    rating_column = f"{each_type}_rating"
    
    #add columns for each type we looking up
    zip_group_df[count_column] = ""
    zip_group_df[rating_column] = ""


    # Use the lat/lng we recovered to search for businesses
    for index, row in zip_group_test_df.iterrows():

        rating_sum = 0
        rating_count = 0

        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

        # change location each iteration while leaving original params in place
        #params["location"] = f"32.8326,-96.7976" #testing 1 lat, lng
        params["location"] = f"{lat},{lng}"

        #Google Places Search
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # make request and print url
        search_results = requests.get(base_url, params=params)

        # convert to json
        search_results = search_results.json()
        #print(json.dumps(search_results, indent=4, sort_keys=True))

        results_list = search_results['results']

        #Set the business count
        zip_group_df.loc[index, count_column] = len(results_list)

        #Loop through results list to get rating for each 
        for each_result in results_list:
            #check for KeyError since not all business have a rating
            try:
                #print(f'{each_result["name"]}: {each_result["rating"]}')
                rating = each_result["rating"]
                rating_count += 1
                rating_sum += rating
            except(KeyError):
                next
        #Set Rating to 0 if there are not businesses returned
        try:
            zip_group_df.loc[index, rating_column] = rating_sum / rating_count
        except(ZeroDivisionError):
            zip_group_df.loc[index, rating_column] = 0
            
        detail_df = pd.DataFrame.from_dict(results_list, orient='columns')

In [10]:
zip_group_df.sort_values('MeanPropertyValue')

NameError: name 'zip_group_df' is not defined

In [12]:
# Store latitude and longitude in locations
#top_5_zip_locations = zip_group_df[["Lat", "Lng"]].
top_5_zip_locations = [(32.8326, -96.7976),(30.26, -97.74),(46.87, -96.78),(47.60, -122.33),(32.71, -117.16)]

In [13]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [14]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(top_5_zip_locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…